In [1]:
#import the necessary libraires
import random
import json
import pickle
import numpy as np
import tensorflow as tf
import nltk

In [2]:
#downloading the nltk packages
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MADHURI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MADHURI\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

In [4]:
# load the intents file
intents = json.loads(open('intents.json').read())

In [5]:
# create empty lists for words classes and documents
words = []
classes = []
documents = []

In [6]:
# create a list of the symbols to ignore
ignoreLetters = ['?', '!', '.', ',']

In [7]:
# looping throught the intents and extracting the features
for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)

        documents.append((wordList, intent['tag']))

        # add tags into the classes file if they are not present
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [8]:
# lemmatize the words and remove duplicates and ingored letters
words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]

In [9]:
words = sorted(set(words))
classes = sorted(set(classes))

In [10]:
# dump these classes and words into pickle files
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [11]:
# create a training set

training = []
empty_output = [0] * len(classes)

In [12]:
# create a bag of words representation of the documents

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]

    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

In [13]:
# create the output row
output_row = list(empty_output)
output_row[classes.index(document[1])] = 1
training.append(bag + output_row)

In [14]:
# shuffle the data
random.shuffle(training)
training = np.array(training)

In [15]:
# create X and Y for training
X_train = training[:, :len(words)]
y_train = training [:, len(words):]

In [16]:
# create a sequntial model using tensorflow
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(y_train[0]), activation='softmax'))

In [17]:
# compile the above model

sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [18]:
# train the model
history = model.fit(np.array(X_train), np.array(y_train), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
1/1 [==============================] - 1s 693ms/step - loss: 4.6186 - accuracy: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 0s 8ms/step - loss: 4.4974 - accuracy: 0.0000e+00
Epoch 3/200
1/1 [==============================] - 0s 8ms/step - loss: 4.3777 - accuracy: 0.0000e+00
Epoch 4/200
1/1 [==============================] - 0s 10ms/step - loss: 4.1951 - accuracy: 0.0000e+00
Epoch 5/200
1/1 [==============================] - 0s 8ms/step - loss: 4.1004 - accuracy: 0.0000e+00
Epoch 6/200
1/1 [==============================] - 0s 8ms/step - loss: 3.8592 - accuracy: 1.0000
Epoch 7/200
1/1 [==============================] - 0s 10ms/step - loss: 4.0230 - accuracy: 0.0000e+00
Epoch 8/200
1/1 [==============================] - 0s 8ms/step - loss: 3.5603 - accuracy: 1.0000
Epoch 9/200
1/1 [==============================] - 0s 16ms/step - loss: 3.4535 - accuracy: 1.0000
Epoch 10/200
1/1 [==============================] - 0s 9ms/step - loss: 3.1070 - accuracy: 1.0000


In [19]:
# save the model
model.save('pikart_customer_service_chatbot.h5', history)
print('done')

done


c:\Users\MADHURI\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
